# DAQ-card data analysis

### This script analyzes the recorded serial output from the daq card.


In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from bisect import bisect_left
import itertools
import time
import datetime

In [26]:
# variables for configuration

data_file = "20171120-1305_DAQ_1800_sec_threshs_300-289-292-288_detector_mode_1E_det1_slab1.txt"
#data_file = "20171109-1219_DAQ_3000_sec_threshs_70-14-79-83_detector_mode_1E.txt"
#data_file = "20171110-1019_DAQ_3000_sec_threshs_70-14-70-83_detector_mode_1E.txt"
#data_file = "20171110-1118_DAQ_1800_sec_threshs_70-14-60-83_detector_mode_1E.txt"
#data_file = "20171110-1140_DAQ_1800_sec_threshs_70-14-50-83_detector_mode_1E.txt"

# the V1 pulse is 2.5 ms long
# the one from the muon hunter is 1 ms long
max_coincidence_time_s = 100*10**(-9)  # 100 ns


# detector areas in cm^2
detector_area = {
                1: 400.0,
                2: 400.0,
                3: 400.0,
                }
max_Hz = 10

In [27]:
# Functions
def simplecount(filename):
    lines = 0
    for line in open(filename, 'U'):
        lines += 1
    return lines

def check_line_for_corruption(line):
    # check for obvious corruption
    if len(line.split(' ')) is not 16:
        return True
    # split line
    split_line = line.split(" ")
    # check each field for currupted-ness
    for i in range(len(split_line)):
        length = len(split_line[i])
        if i in (0,9):
            if length != 8:
                return True
        if i in range(1,9):
            if length != 2:
                return True
        if i == 10:
            if length != 10:
                return True
        if i == 11:
            if length != 6:
                return True
        if i == 12:
            if length != 1:
                return True
        if i == 13:
            if length != 2:
                return True
        if i == 14:
            if length != 1:
                return True
        if i == 15:
            if length != 6:
                return True
        
    # all checks passed
    return False

def findClosest(list_to_search, value):
    # bisect only works because we know that our list is sorted
    pos = bisect_left(list_to_search, value)
    if pos == 0:
        return list_to_search[0]
    if pos == len(list_to_search):
        return list_to_search[-1]
    before = list_to_search[pos - 1]
    after = list_to_search[pos]
    if after - value < value - before:
       return after
    else:
       return before

def parse_edge_data_forgetfull_parser(file_lines):
    edge_and_tirgger_pos = [[] for i in range(9)]

    previous_PPS = -1000
    current_PPS = -1000

    rollover_counter_trig = 0
    rollover_counter_pps = 0
    old_unadjusted_time_trig = 0
    old_unadjusted_time_pps = 0

    # list for saving our data
    edge_and_tirgger_pos = [[] for i in range(9)]

    lines_to_read = len(file_lines)
    print("Lines to read: "+ str(lines_to_read))

    print("Starting to parse section")
    line_count = 0
    discarded_lines = 0
    line_buffer = []
    event_lengths = []
    discarded_event_lengths = []

    for into_buffer_line in file_lines:
        line_count += 1
        if (line_count % (lines_to_read//20)) == 0:
            print("Percentage of section parsed: " + str(round( float(line_count)*100 / float(lines_to_read), 2)))
        # buffer lines to make sure the event is not corrupted
        # empty the buffer when a corrupt line was found
        if check_line_for_corruption(into_buffer_line):
            discarded_lines += 1 + len(line_buffer)
            discarded_event_lengths.append(len(line_buffer)+1)
            line_buffer = []
            continue
        # if the buffer was empty add the line and continue
        if len(line_buffer) == 0:
            line_buffer.append(into_buffer_line)
            continue
        # if the event has come to an end; parse it, otherwise just buffer the line
        new_timestamp = int(into_buffer_line.split(" ")[0], 16)
        old_timestamp = int(line_buffer[0].split(" ")[0], 16)
        if new_timestamp == old_timestamp:
            line_buffer.append(into_buffer_line)    
        else:
            event_lengths.append(len(line_buffer))
            for current_line in line_buffer:
                # split line
                split_line = current_line.split(" ")        

                # parse the line
                try:
                    # get the current pps and trigger
                    pps = int(split_line[9], 16)
                    trigger_clock = int(split_line[0], 16)
                    # check for rollovers
                    if old_unadjusted_time_pps > pps:
                        rollover_counter_pps += 1
                    if old_unadjusted_time_trig > trigger_clock:
                        rollover_counter_trig += 1
                    # adjust for rollovers
                    old_unadjusted_time_pps = pps
                    pps = rollover_counter_pps * 2**36 + pps
                    old_unadjusted_time_trig = trigger_clock
                    trigger_clock = rollover_counter_trig * 2**36 + trigger_clock

                    # save pps
                    if pps != current_PPS:
                        previous_PPS = current_PPS
                        current_PPS = pps
                    # check if we already have a previous pps
                    if previous_PPS < 0:
                        continue

                    # calculate the UTC time
                    day = int(split_line[11][0:2])
                    month = int(split_line[11][2:4])
                    year = int(split_line[11][4:6])
                    hour = int(split_line[10][0:2])
                    minute = int(split_line[10][2:4])
                    second = float(split_line[10][4:10])
                    second = int(round(second + float(split_line[15])/1000 ))
                    time_from_gps = datetime.datetime(year+2000,
                                                    month,
                                                    day,
                                                    hour,
                                                    minute,
                                                    second,
                                                    tzinfo=None)
                    unix_seconds = (time_from_gps - datetime.datetime(1970,1,1,tzinfo=None)).total_seconds()
                    clock_rate = current_PPS - previous_PPS
                    sub_sec = float(trigger_clock - current_PPS) / float(clock_rate)    

                    # convert stuff into the binary representation
                    bin_rep = []
                    for i in range(1,9):
                        bin_rep.append(bin(int(split_line[i],16))[2:].zfill(8))

                    # check if the daq triggered
                    triggered = False
                    if bin_rep[0][0] == '1':
                        triggered =  True
                    if triggered:
                        edge_and_tirgger_pos[0].append(unix_seconds+sub_sec)

                    # check rising and falling edges
                    for i in range(len(bin_rep)):
                        bin_number = bin_rep[i]
                        if bin_number[2] == '1':
                            edge_nano_secs = float(int(bin_number[3:],2))/32 * 24
                            edge_sub_secs = edge_nano_secs * 10**(-9)
                            edge_and_tirgger_pos[i+1].append(unix_seconds+sub_sec+edge_sub_secs)
                except ValueError:
                    print("Caught a value error, that had slipped through the corruption check. Line: "+ str(line_count))
                    print("Errorous line: "+ current_line)
                    discarded_lines += 1
                    break
            # empty the buffer and pre-save the next line
            line_buffer = []
            line_buffer.append(into_buffer_line)  

    print("Percentage of lines that were corrputed: "+str(100.0*discarded_lines/float(line_count)))
    print("Average event length: {:.2f}".format(np.mean(event_lengths)))
    print("Median event length: {:.0f}".format(np.median(event_lengths)))
    print("Stdabw. event length: {:.2f}".format(np.std(event_lengths)))
    print("Average discarded event length: {:.2f}".format(np.mean(discarded_event_lengths)))
    print("Median discarded event length: {:.0f}".format(np.median(discarded_event_lengths)))
    print("Stdabw. discarded event length: {:.2f}".format(np.std(discarded_event_lengths)))
    return edge_and_tirgger_pos

def calculate_correlations_from_edge_data(edge_data, max_coincidence_time_s, active_channels=[1,2,3]):
    # calculate correlations

    combinations = list(itertools.combinations(detector_area.keys(), 2))
    correlations_to_do = []
    for i in range(len(detector_area.keys())):
        correlations_to_do.append((combinations[i], detector_area.keys()[2-i]))

    correlations = {}
    for corr_tupel in correlations_to_do:
        (key1, key2), key3 = corr_tupel
        # do the first correlation
        base_correlations = {'base_time':[], 'diff':[]}

        for key1_time in edge_data[key1*2+1]:
            closest_key2_time = findClosest(edge_data[key2*2+1], key1_time)
            diff = closest_key2_time - key1_time
            if (abs(diff) <= max_coincidence_time_s):
                base_correlations['base_time'].append(key1_time)
                base_correlations['diff'].append(diff)
        correlations["CH"+str(key1)+"<->CH"+str(key2)] = base_correlations

        second_level_correlation = {'base_time':[], 'diff':[]}
        # do second correlation
        for base_time in base_correlations['base_time']:
            closest_key3_time = findClosest(edge_data[key3*2+1], base_time)
            diff = closest_key3_time - base_time
            if (abs(diff) <= max_coincidence_time_s):
                second_level_correlation['base_time'].append(base_time)
                second_level_correlation['diff'].append(diff)

        correlations["[CH"+str(key1)+"<->CH"+str(key2)+"]<->CH"+str(key3)] = second_level_correlation
    
    return correlations

In [28]:
lines_to_parse = []
for line in open(data_file, 'U'):
    lines_to_parse.append(line)

edge_and_tirgger_pos = parse_edge_data_forgetfull_parser(lines_to_parse)

Lines to read: 26799
Starting to parse section
Percentage of section parsed: 5.0
Percentage of section parsed: 9.99
Percentage of section parsed: 14.99
Percentage of section parsed: 19.99
Percentage of section parsed: 24.98
Percentage of section parsed: 29.98
Percentage of section parsed: 34.98
Percentage of section parsed: 39.97
Percentage of section parsed: 44.97
Percentage of section parsed: 49.96
Percentage of section parsed: 54.96
Percentage of section parsed: 59.96
Percentage of section parsed: 64.95
Percentage of section parsed: 69.95
Percentage of section parsed: 74.95
Percentage of section parsed: 79.94
Percentage of section parsed: 84.94
Percentage of section parsed: 89.94
Percentage of section parsed: 94.93
Percentage of section parsed: 99.93
Percentage of lines that were corrputed: 0.00746296503601
Average event length: 1.50
Median event length: 1
Stdabw. event length: 0.66
Average discarded event length: 1.00
Median discarded event length: 1
Stdabw. discarded event length:

In [29]:
for i in range(len(edge_and_tirgger_pos)):
    print(i, len(edge_and_tirgger_pos[i]))

(0, 9234)
(1, 0)
(2, 0)
(3, 8608)
(4, 6992)
(5, 8171)
(6, 5222)
(7, 8099)
(8, 5067)


In [30]:
corrs = calculate_correlations_from_edge_data(edge_and_tirgger_pos, max_coincidence_time_s, active_channels=[1,2,3])

keys = sorted(corrs.keys())
for key in keys:
    print(key+": {:d}".format(len(corrs[key]['diff'])))
    if key[0] == '[':
        print("Percentage of "+key[1:10]+": {:.2f}".format(100.0*len(corrs[key]['diff'])/len(corrs[key[1:10]]['diff'])))

CH1<->CH2: 7327
CH1<->CH3: 7295
CH2<->CH3: 6860
[CH1<->CH2]<->CH3: 6167
Percentage of CH1<->CH2: 84.17
[CH1<->CH3]<->CH2: 6167
Percentage of CH1<->CH3: 84.54
[CH2<->CH3]<->CH1: 6157
Percentage of CH2<->CH3: 89.75


In [31]:
# calculate test runntime
min_s_time = edge_and_tirgger_pos[0][0]
max_s_time = edge_and_tirgger_pos[0][len(edge_and_tirgger_pos[0])-1]
s_time = (max_s_time - min_s_time)
print(s_time, s_time/60, max_s_time, min_s_time)

(1798.394207239151, 29.973236787319184, 1511181344.7552507, 1511179546.3610435)


In [19]:
# tell statistics
print("Maximum coincidence time [s]: {:.4g}".format(max_coincidence_time_s))
print("Test runtime [min]: " + str(s_time/60))
print("")

for key in detector_area.keys():
    print("CH"+str(key)+" statistics:")
    print("\tDetector area [cm^2]: " + str(detector_area[key]))
    print("\tTotal number of events: " + str(len(edge_and_tirgger_pos[key*2+1])))
    print("\tEfficency [events/cm^2/min]: " + str(len(edge_and_tirgger_pos[key*2+1])/detector_area[key]/(s_time/60)))
    print("")

keys = reversed(sorted(corrs.keys()))
for key in keys:
    print("Coincidence "+key)

    print("\tTotal number of coincidences: " +str(len(corrs[key]['diff'])))
    print("\tEfficency [events/cm^2/min]: "+str(len(corrs[key]['diff'])/detector_area[1]/(s_time/60)))
    
    print("\tTime between events for one coincidence:")
    print("\t\tMedian [s]: {:.4g}".format(np.median(corrs[key]['diff'])))
    print("\t\tAverage [s]: {:.4g}".format(np.mean(corrs[key]['diff'])))
    print("\t\tStandard deviation [s]: {:.4g}".format(np.std(corrs[key]['diff'])))
    print("")

Maximum coincidence time [s]: 1e-07
Test runtime [min]: 19.9758048654

CH1 statistics:
	Detector area [cm^2]: 400.0
	Total number of events: 5197
	Efficency [events/cm^2/min]: 0.650411840102

CH2 statistics:
	Detector area [cm^2]: 400.0
	Total number of events: 6339
	Efficency [events/cm^2/min]: 0.793334742045

CH3 statistics:
	Detector area [cm^2]: 400.0
	Total number of events: 5578
	Efficency [events/cm^2/min]: 0.698094524551

Coincidence [CH2<->CH3]<->CH1
	Total number of coincidences: 4302
	Efficency [events/cm^2/min]: 0.538401334639
	Time between events for one coincidence:
		Median [s]: 0
		Average [s]: 0
		Standard deviation [s]: 0

Coincidence [CH1<->CH3]<->CH2
	Total number of coincidences: 4290
	Efficency [events/cm^2/min]: 0.536899517806
	Time between events for one coincidence:
		Median [s]: 0
		Average [s]: 0
		Standard deviation [s]: 0

Coincidence [CH1<->CH2]<->CH3
	Total number of coincidences: 4290
	Efficency [events/cm^2/min]: 0.536899517806
	Time between events for 